In [32]:
"""
Bayesian spam filter
"""

'\nBayesian spam filter\n'

In [10]:
import pandas as pd
import numpy as np


In [33]:
data = pd.read_csv('./data/train_data.csv')

In [31]:
data.iloc[24]

word_freq_make                    0
word_freq_address                 0
word_freq_all                  0.33
word_freq_3d                      0
word_freq_our                     0
word_freq_over                 0.49
word_freq_remove                  0
word_freq_internet             1.32
word_freq_order                0.16
word_freq_mail                 5.12
word_freq_receive                 0
word_freq_will                    0
word_freq_people                  0
word_freq_report               0.66
word_freq_addresses               0
word_freq_free                    0
word_freq_business             0.33
word_freq_email                   0
word_freq_you                  0.33
word_freq_credit                  0
word_freq_your                    0
word_freq_font                    0
word_freq_000                     0
word_freq_money                   0
word_freq_hp                      0
word_freq_hpl                     0
word_freq_george                  0
word_freq_650               

[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
 5.55 0.0 0.0 0.0 0.0 0.0 0.0 0.0 2.77 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
 1.0 1 7 True 1103]


In [296]:
sample_email = "make, address, 3d make FSERAS conference conference 1999 GREADER tragujsdkgsfo gfsgodhfguio fhguid fhgudfigufghdfui ghd hudihgihdfi huguidf hfdiguidf. iudfiguhdf gidfuhgdfhdfuigdif. guiduifhmake, address, table, conference, !!!!! directi  DSDASD"
words = []
chars = []

for c in data.columns[0:48]:
    words.append(c[c.index("q_")+2:])

for c in data.columns[48:54]:
    chars.append(c[c.index("q_")+2:])
    

def parse_email(text):
    wf = np.zeros(48)
    cf = np.zeros(6)
    crl_a = 0
    crl_max = 0
    crl_t = 0
    total_words = 0
    total_chars = len(text)
    capital_words = 0
    
    text = text.split(' ')
    total_words = len(text)
    wds = []
    for t in text:
        # das capitals
        
        cap_run = 0 #capital run length
        for i in range(len(t)):
            if t[i].isupper():
                cap_run +=1
        
        if cap_run > 0:
            capital_words +=1
        # check if current word length's cap len is greater than the max
        if cap_run > crl_max: 
            crl_max = cap_run
        
        crl_t += cap_run
        # now back to the lcase
        t = t.lower()
        
        #   das special characters
        if len(t) > 0:
            if t[0] in chars:
                cf[chars.index(t[0])] += 1
            elif t[len(t)-1] in chars:
                cf[chars.index(t[len(t)-1])] += 1
            
            if t[0].isalpha() != True and t[0].isnumeric() != True:
                t = t[1:]
            elif t[len(t)-1].isalpha() != True and t[len(t)-1].isnumeric() != True:
                t = t[:-1]
                
        if t in words:
            wf[words.index(t)] += 1

    wf = (wf  * 100 / total_words);
    cf = (cf * 100 / total_chars)
    crl_a = crl_t / capital_words
    

    res = wf.tolist() + cf.tolist()
    res.extend([crl_a, crl_max, crl_t])
    return res
    

f = parse_email(sample_email)
f = np.array(f).reshape((57, 1))
thetas = np.random.randn(58,1)




Y = data[['ham']].to_numpy().ravel()
darr = data.drop(['ham', 'Id'], axis=1)

X = darr.to_numpy()
m = len(X)

X = np.c_[np.zeros((len(X), 1)), X.reshape((len(X), 57))]

print(X.shape)
print(thetas.shape)
print(Y.shape)

n = 1000
eta = 0.01

from sklearn.linear_model import LogisticRegression

def sigmoid(x):
    return np.where(x >= 0, 
                    1 / (1 + np.exp(-x)), 
                    np.exp(x) / (1 + np.exp(x)))


for c in range(10):
    for i in range(n):
        ri = np.random.randint(m)
        xi = X[ri:ri+1]
        yi = Y[ri:ri+1]
        prod = thetas.dot(xi)
        grad = xi.T.dot(sigmoid(thetas.T.dot(xi.T)) - yi)
        thetas = thetas - eta*grad
        

        

(3680, 58)
(58, 1)
(3680,)


<ipython-input-296-93017f2308f2>:94: RuntimeWarning: overflow encountered in exp
  1 / (1 + np.exp(-x)),
<ipython-input-296-93017f2308f2>:95: RuntimeWarning: overflow encountered in exp
  np.exp(x) / (1 + np.exp(x)))
<ipython-input-296-93017f2308f2>:95: RuntimeWarning: invalid value encountered in true_divide
  np.exp(x) / (1 + np.exp(x)))


In [280]:
log_reg = LogisticRegression()
log_reg.fit(X, Y)

/usr/local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [307]:
res = parse_email(sample_email)

In [308]:
res = np.array(res).reshape(1, 57)
res = np.c_[0, res]

In [329]:
log_reg.predict(res)

array([ True])

In [332]:
y = sigmoid(thetas.T.dot(X[25].T))

<ipython-input-296-93017f2308f2>:94: RuntimeWarning: overflow encountered in exp
  1 / (1 + np.exp(-x)),


In [333]:
y

array([0.])